In [ ]:
import re
import csv
import time
from playwright.async_api import async_playwright
from parsel import Selector

In [ ]:
playwright = await async_playwright().start()

browser = await playwright.chromium.launch(
    headless=False,
    channel="chrome",
    args=["--disable-blink-features", "--disable-blink-features=AutomationControlled"],
)


context = await browser.new_context(
    user_agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
)

page = await context.new_page()
page.set_default_timeout(3000)




def extract_lat_long(url):
    pattern = r'@(-?\d+\.\d+),(-?\d+\.\d+)'
    match = re.search(pattern, url)
    if match:
        lat, long = match.groups()
        lat, long = float(lat), float(long)
        return lat, long
    else:
        return None, None



await page.goto("https://www.google.com/maps")


print("========== Code executed ============")

In [3]:
urls_data = []

with open('120.csv', 'r') as f: 
    reader = csv.DictReader(f)
    for row in reader:
        _01_category = row['_01_category']
        _02_state = row['_02_state']
        _03_branch_name = row['_03_branch_name']
        _04_address = row['_04_address']
        # _05_latitude
        # _06_longitude
        _07_bank_type = row['_09_bank_type']
        _08_mother_bank_name = row['_11_mother_bank_name']

        _07_google_map_url = row['_07_google_map_url']

        urls_data.append({
            "_01_category": _01_category,
            "_02_state": _02_state,
            "_03_branch_name": _03_branch_name,
            "_04_address": _04_address,
            "_07_bank_type": _07_bank_type,
            "_08_mother_bank_name": _08_mother_bank_name,
            "_07_google_map_url": _07_google_map_url
        })

print(len(urls_data))
# test_urls_data = urls_data[334:]
# print(len(test_urls_data))

101


In [4]:
data = []
count = 1

for row in urls_data:
    _01_category = row['_01_category']
    _02_state = row['_02_state']
    _03_branch_name = row['_03_branch_name']
    _04_address = row['_04_address']
    _07_bank_type = row['_07_bank_type']
    _08_mother_bank_name = row['_08_mother_bank_name']
    _07_google_map_url = row['_07_google_map_url']

    

    # Playwright code
    try:
        await page.goto(_07_google_map_url)
    except:
        g = 5
    await page.wait_for_timeout(1000)
    _05_latitude, _06_longitude = extract_lat_long(page.url)


    meta_dic = {
        "_01_category": _01_category,
        "_02_state": _02_state,
        "_03_branch_name": _03_branch_name,
        "_04_address": _04_address,
        "_05_latitude": _05_latitude,
        "_06_longitude": _06_longitude,
        "_07_bank_type": _07_bank_type,
        "_08_mother_bank_name": _08_mother_bank_name,
    }

    data.append(meta_dic)

    print(f"\rProcessing profile {count}...", end="")
    count += 1
    

Processing profile 101...

In [5]:
csv_filename = 'lat-long-2.csv'

field_names = data[0].keys() if data else []
with open(csv_filename, 'a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(data)